In [163]:
import time 
import multiprocessing as mp
from multiprocessing import Pool
from collections import defaultdict
import sys
sys.path.append('/home/ak/Documents/Research/PaperCode/stylised_facts')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import lob_for_futures as lobFut
from lob_for_futures import *

from scipy.stats import jarque_bera
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
import pingouin as pig
import seaborn as sns
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import pickle as pkl
import matplotlib.ticker as mtick

from functools import partial

# Lets create a function that shifted your dataframe first before calling the corr().
# Let us break down what we hope to accomplish, and then translate that into code.
# For each hour (row) and for a given feature (column) I would like to find the value
# for that feature N hours prior.
# For each value of N (1-6 in our case) I want to make a new column
# for that feature representing the Nth prior hour's measurement.

elements = '/media/ak/Elements/OrderBookProcessedData'
experimentsLocation = '/media/ak/T7/June4th2022Experiments'
bars =['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
plt.style.use(os.path.join('/home/ak/.config/matplotlib','latexstyle3.mplstyle'))
figures_destination = '/home/ak/Documents/Research/Papers/figures/'

jb_dict = dict()
standarised_returns = defaultdict(dict)
dataFolder = lobFut.dataFolder
quotes = [f for f in os.listdir(dataFolder) if str('_quotes') in f]
trades = [f for f in os.listdir(dataFolder) if str('_trades') in f]

In [17]:
def print_cube(num):
    """
    function to print cube of given num
    """
    print("Cube: {}".format(num * num * num))


In [18]:
 
def print_square(num):
    """
    function to print square of given num
    """
    print("Square: {}".format(num * num))


In [20]:


# if __name__ == '__main__':
symbolIdx = 0
symbols = sorted(os.listdir(elements))
procsdSymbolFolder = os.path.join(elements, symbols[symbolIdx])
files = sorted(os.listdir(procsdSymbolFolder))
bars = ['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
symbol = symbols[symbolIdx]
print(symbol)

#     pickle_out_filename = os.path.join( experimentsLocation, "".join((str(symbol), "_JB_results.pkl")))

#     pickle.dump(jb_dict, open(pickle_out_filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

#     pickle_out_returns = os.path.join(experimentsLocation, "".join((str(symbol), "_standard_returns.pkl")))
#     pickle.dump(standarised_returns, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)






DU1


In [21]:


def jb_calculation(symbolIdx, filesIdx):

    # add location variables. these i have to figure how to abstract away
    print(symbols[symbolIdx])
    procsdSymbolFolder = os.path.join(elements, symbols[symbolIdx])
    print(procsdSymbolFolder)
    files = sorted(os.listdir(procsdSymbolFolder))
    
    fileLocation = os.path.join(procsdSymbolFolder, files[filesIdx])
    print(fileLocation)
    
    # pick the various files
    volume_bar_dict = open_pickle_filepath(fileLocation)[bars[0]]
    calendar_bar_dict = open_pickle_filepath(fileLocation)[bars[1]]
    usd_volume_bar_dict = open_pickle_filepath(fileLocation)[bars[2]]
    tick_bar_dict = open_pickle_filepath(fileLocation)[bars[3]]
    # get the dataframes

    volume_bar_df = volume_bar_dict[list(volume_bar_dict.keys())[0]]
    calendar_bar_df = calendar_bar_dict[list(calendar_bar_dict.keys())[0]]
    usd_volume_df = usd_volume_bar_dict[list(usd_volume_bar_dict.keys())[0]]
    tick_bar_df = tick_bar_dict[list(usd_volume_bar_dict.keys())[0]]
    # returns

    vb_ret = returns(volume_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # volume
    tb_ret = returns(tick_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # tick
    usdvb_ret = returns(usd_volume_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # usd volume
    cb_ret = returns(calendar_bar_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # calendar
    # calculating JB statistic
    jb_value_tick, _ = jarque_bera(tb_ret)
    jb_value_vol, _ = jarque_bera(vb_ret)
    jb_value_dollar, _ = jarque_bera(usdvb_ret)
    jb_value_calendar, _ = jarque_bera(cb_ret)


    jb_test_df = pd.DataFrame(data={'jarque_bera_results': [jb_value_tick,
                                                            jb_value_vol,
                                                            jb_value_dollar,
                                                            jb_value_calendar]
                                    },
                              index=['tick', 'vol', 'dollar', 'calendar'])
    pickle_out_returns = os.path.join(experimentsLocation, "".join((str(symbols[symbolIdx]), "_"+str(filesIdx)+"_jb_stats.pkl")))
    pickle.dump(jb_test_df, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

    print("produced and saved JB stats for :", symbols[symbolIdx], filesIdx)


    

In [22]:
jb_calculation(4, 4)
os.listdir('/media/ak/Elements/OrderBookProcessedData/')

RX1
/media/ak/Elements/OrderBookProcessedData/RX1
/media/ak/Elements/OrderBookProcessedData/RX1/RX1_20180417_consolidated_lob_clock_tick_100_cal_resample_600.pkl
produced and saved JB stats for : RX1 4


['RX1', 'DU1', 'JB1', 'KE1', 'OE1', 'TU1', 'US1']

In [23]:
def standarised_returns(symbolIdx, filesIdx):
    standarised_returns = defaultdict(dict)
    # add location variables. these i have to figure how to abstract away
    ### all files here ###
    procsdSymbolFolder = os.path.join(elements, symbols[symbolIdx])
    files = sorted(os.listdir(procsdSymbolFolder))
    fileLocation = os.path.join(procsdSymbolFolder, files[filesIdx])
    
    volume_bar_dict = open_pickle_filepath(fileLocation)[bars[0]]
    calendar_bar_dict = open_pickle_filepath(fileLocation)[bars[1]]
    usd_volume_bar_dict = open_pickle_filepath(fileLocation)[bars[2]]
    tick_bar_dict = open_pickle_filepath(fileLocation)[bars[3]]
    ### all dataframes here ###

    volume_bar_df = volume_bar_dict[list(volume_bar_dict.keys())[0]]
    calendar_bar_df = calendar_bar_dict[list(calendar_bar_dict.keys())[0]]
    usd_volume_df = usd_volume_bar_dict[list(usd_volume_bar_dict.keys())[0]]
    tick_bar_df = tick_bar_dict[list(usd_volume_bar_dict.keys())[0]]


    vb_ret = returns(volume_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # volume
    tb_ret = returns(tick_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # tick
    usdvb_ret = returns(usd_volume_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # usd volume
    cb_ret = returns(calendar_bar_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # calendar
    
    tick_standard = (tb_ret - tb_ret.mean()) / tb_ret.std()
    volume_standard = (vb_ret - vb_ret.mean()) / vb_ret.std()
    dollar_standard = (usdvb_ret - usdvb_ret.mean()) / usdvb_ret.std()
    cb_standard = (cb_ret - cb_ret.mean()) / cb_ret.std()
    standarised_returns['tick'][filesIdx] = tick_standard
    standarised_returns['calendar'][filesIdx] = cb_standard
    standarised_returns['volume'][filesIdx] = volume_standard
    standarised_returns['dollar'][filesIdx] = dollar_standard
    
    pickle_out_returns = os.path.join(experimentsLocation, "".join((str(symbol), "_"+str(filesIdx)+"_standard_returns.pkl")))
    pickle.dump(standarised_returns, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    return standarised_returns




In [24]:
test =standarised_returns(1, 1)

In [25]:
test['tick']

{1: tick_bucket
 2.0    -0.206286
 3.0     1.928834
 4.0    -1.234174
 5.0    -2.713881
 6.0     1.362019
 7.0    -0.081969
 8.0    -1.638321
 9.0     1.832676
 10.0    1.073569
 11.0    1.054399
 12.0    0.208031
 13.0    1.613614
 14.0    0.211519
 15.0   -1.939518
 16.0    0.064161
 17.0    0.214410
 18.0    0.056166
 19.0    0.201806
 20.0   -0.018024
 21.0   -0.038553
 22.0    0.226089
 23.0    0.269472
 24.0    0.148403
 25.0   -0.290909
 26.0    0.044085
 27.0   -0.107348
 28.0   -0.083489
 29.0    1.782219
 30.0    0.107197
 31.0    0.142553
 32.0   -0.302311
 33.0    0.008324
 34.0   -0.012104
 35.0    0.151524
 36.0   -0.004914
 37.0    0.045771
 38.0    0.043841
 39.0    0.020466
 40.0    0.026410
 41.0    0.130030
 42.0   -0.094306
 43.0   -0.037115
 44.0   -0.046665
 45.0    0.137669
 46.0    0.045606
 47.0    0.010209
 48.0   -0.292678
 49.0    0.236629
 50.0    1.837103
 51.0   -2.134190
 52.0   -3.044473
 53.0    0.331837
 54.0   -1.245412
 dtype: float64}

In [42]:
# if __name__ == "__main__":
#     # creating processes
#     p1 = multiprocessing.Process(target=jb_calculation, args=(1, ))
    
 
#     # starting process 1
#     p1.start()
#     # starting process 2
# #     p2.start()
 
#     # wait until process 1 is finished
# #     p1.join()
#     # wait until process 2 is finished
# #     p2.join()
 
#     # both processes finished
#     print("Done!")

In [ ]:
# from multiprocessing import Pool
# with Pool(5) as p:
#     print(p.map(jb_calculation, [1, 2, 3, 5, 6 ,7, 8, 25]))

In [26]:
from functools import partial
from itertools import repeat
from multiprocessing import Pool, freeze_support

In [ ]:

def main():
    a_args = [1, 2]
    second_arg = [1, 2, 3, 4, 5, 6]
    with Pool() as pool:
#        L = pool.starmap(standarised_returns, [(1, 1), (1, 2), (1, 3)])
        K = pool.starmap(standarised_returns, list(itertools.product(a_args, second_arg)))
#         M = pool.starmap(jb_calculation, zip(a_args, second_arg))
#         N = pool.map(partial(jb_calculation, second_arg), a_args)
#         assert L == K

if __name__=="__main__":



In [39]:
a_args = [1]
second_arg = [f for f in range(0, 20, 1)]

In [41]:
# freeze_support()
# with Pool() as pool:
#     L = pool.starmap(standarised_returns, list(itertools.product(a_args, second_arg)))
#     K = pool.starmap(jb_calculation, list(itertools.product(a_args, second_arg)))

In [ ]:
print(L)

In [ ]:
list(zip(itertools.product(FIRST, SECOND)))

In [ ]:
test_dict =open_pickle_filepath('/media/ak/T7/June4th2022Experiments/DU1_standard_returns.pkl')

In [ ]:
test_dict.keys()

In [46]:
files =[f for f in os.listdir('/media/ak/T7/June4th2022Experiments/') if str('_standard_returns') in f]

In [172]:
symbol_ =' OE1'
type_file ='_standard_returns'
symbol_type_files = [f for f in files if str(symbol) in f if str(type_file) in f]
bars = ['tick', 'calendar', 'volume', 'dollar']
symbol_type_files

['DU1_19_standard_returns.pkl',
 'DU1_18_standard_returns.pkl',
 'DU1_17_standard_returns.pkl',
 'DU1_16_standard_returns.pkl',
 'DU1_15_standard_returns.pkl',
 'DU1_14_standard_returns.pkl',
 'DU1_13_standard_returns.pkl',
 'DU1_12_standard_returns.pkl',
 'DU1_11_standard_returns.pkl',
 'DU1_10_standard_returns.pkl',
 'DU1_9_standard_returns.pkl',
 'DU1_8_standard_returns.pkl',
 'DU1_7_standard_returns.pkl',
 'DU1_6_standard_returns.pkl',
 'DU1_0_standard_returns.pkl',
 'DU1_5_standard_returns.pkl',
 'DU1_4_standard_returns.pkl',
 'DU1_3_standard_returns.pkl',
 'DU1_1_standard_returns.pkl',
 'DU1_2_standard_returns.pkl']

In [178]:
list(oaded_returns['tick'].keys())[0]

NameError: name 'oaded_returns' is not defined

In [170]:
list(symbol_type_files

['DU1_19_standard_returns.pkl',
 'DU1_18_standard_returns.pkl',
 'DU1_17_standard_returns.pkl',
 'DU1_16_standard_returns.pkl',
 'DU1_15_standard_returns.pkl',
 'DU1_14_standard_returns.pkl',
 'DU1_13_standard_returns.pkl',
 'DU1_12_standard_returns.pkl',
 'DU1_11_standard_returns.pkl',
 'DU1_10_standard_returns.pkl',
 'DU1_9_standard_returns.pkl',
 'DU1_8_standard_returns.pkl',
 'DU1_7_standard_returns.pkl',
 'DU1_6_standard_returns.pkl',
 'DU1_0_standard_returns.pkl',
 'DU1_5_standard_returns.pkl',
 'DU1_4_standard_returns.pkl',
 'DU1_3_standard_returns.pkl',
 'DU1_1_standard_returns.pkl',
 'DU1_2_standard_returns.pkl']

In [98]:
file_index = 1
file_path =os.path.join(experimentsLocation, symbol_type_files[file_index])
loaded_returns = open_pickle_filepath(file_path)
index_item = list(loaded_returns[bars[0]].keys())[0]

In [99]:
tr = loaded_returns['tick'][index_item]
calr = loaded_returns['calendar'][index_item]
vr = loaded_returns['volume'][index_item]
dr = loaded_returns['dollar'][index_item]




In [131]:

tick_dict = dict()
usd_dict = dict()
calendar_dict = dict()
volm_dict = dict()
for file_index, _ in enumerate(symbol_type_files):
    file_path =os.path.join(experimentsLocation, symbol_type_files[file_index])
    loaded_returns = open_pickle_filepath(file_path)
    index_item = list(loaded_returns[bars[0]].keys())[0]
    tick_dict[file_index+1]= (pd.DataFrame(loaded_returns['tick'][index_item].values))
    usd_dict[file_index+1]= (pd.DataFrame(loaded_returns['dollar'][index_item].values))
    calendar_dict[file_index+1]= (pd.DataFrame(loaded_returns['calendar'][index_item].values))
    volm_dict[file_index+1]= (pd.DataFrame(loaded_returns['volume'][index_item].values))
    
    
    

In [132]:
volm_dict

{1:               0
 0     -0.001223
 1     -0.034204
 2     -0.096343
 3      0.775518
 4      0.019516
 ...         ...
 15451 -0.439982
 15452 -0.064730
 15453 -0.116264
 15454 -0.313538
 15455  1.873759
 
 [15456 rows x 1 columns],
 2:              0
 0     0.000667
 1    -0.023910
 2    -0.070214
 3     0.593014
 4     0.165174
 ...        ...
 6447  0.112401
 6448 -2.815916
 6449 -0.219445
 6450 -0.326284
 6451 -0.134296
 
 [6452 rows x 1 columns],
 3:               0
 0     -0.001223
 1     -0.034204
 2     -0.096343
 3      0.775518
 4      0.019516
 ...         ...
 15451 -0.439982
 15452 -0.064730
 15453 -0.116264
 15454 -0.313538
 15455  1.873759
 
 [15456 rows x 1 columns],
 4:              0
 0     0.000667
 1    -0.023910
 2    -0.070214
 3     0.593014
 4     0.165174
 ...        ...
 6447  0.112401
 6448 -2.815916
 6449 -0.219445
 6450 -0.326284
 6451 -0.134296
 
 [6452 rows x 1 columns],
 5:               0
 0     -0.001223
 1     -0.034204
 2     -0.096343
 3      0.7

In [179]:
user_dict = calendar_dict
def produce_dict(user_dict):
    columns_values = list(user_dict.keys())
    _df =pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                               for i in user_dict.keys() 
                               for j in user_dict[i].keys()},
                           orient='index')
    return_df = _df.reset_index().drop(columns=['level_0', 'level_1'])
    return return_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163
0,-3.035199,0.339819,0.101292,-0.867509,-0.352161,0.386487,-0.114646,0.028338,-0.271671,-0.450577,0.453338,0.426511,-2.734621,0.836345,0.234737,-1.853026,-2.031305,-1.088093,1.312025,-0.086444,-0.114457,0.109626,0.053639,-1.476198,1.763554,0.314331,-0.461516,-0.026144,1.93647,-0.066335,0.121528,0.247095,-1.55651,0.03999,0.088676,-2.152806,0.151234,0.141001,0.362897,1.726878,0.007112,0.044281,-2.376506,0.153878,2.392119,0.08812,-0.236717,0.016733,-0.034499,0.065807,-0.074531,0.047404,0.141387,-0.074853,-0.038261,-0.020196,0.03234,-0.064777,2.125607,-0.045468,0.04781,-0.06545,-0.000193,-0.429509,0.655512,0.099622,-0.092667,-0.442808,0.019282,0.430884,0.045832,-0.03387,0.0709,0.004135,-0.122761,0.018717,-0.038279,2.01171,2.054469,-0.01099,0.101134,0.031641,-2.199975,0.054284,0.020288,0.137378,2.233222,0.080574,-0.123849,-0.358455,2.085705,-1.983462,0.677206,-0.173973,-0.038308,0.372832,0.326945,-0.291275,-0.065318,0.428629,-2.564597,0.345225,2.088677,-2.335162,-0.113657,0.075214,2.059409,0.525848,-0.430789,0.223353,0.432391,-2.635306,0.246727,0.036412,0.046987,0.135662,0.031599,0.052198,0.432797,-2.128332,-0.216767,0.022602,2.751684,0.087278,-2.967879,2.933561,-0.036041,0.067418,0.213866,-0.007657,-1.841815,1.942968,-0.076189,0.052228,0.149901,-0.007161,-0.064303,0.058158,-0.532908,0.429528,0.405651,-0.070227,0.236214,-0.496152,0.045877,0.066543,-0.041307,0.334081,-0.111002,0.102659,-0.027857,0.13548,-0.020582,-0.104633,0.109376,-0.00774,0.115855,-0.025528,0.038041,-0.022756,0.252013,0.195369,-1.334428,0.618779
1,-3.035199,0.339819,0.101292,-0.867509,-0.352161,0.386487,-0.114646,0.028338,-0.271671,-0.450577,0.453338,0.426511,-2.734621,0.836345,0.234737,-1.853026,-2.031305,-1.088093,1.312025,-0.086444,-0.114457,0.109626,0.053639,-1.476198,1.763554,0.314331,-0.461516,-0.026144,1.93647,-0.066335,0.121528,0.247095,-1.55651,0.03999,0.088676,-2.152806,0.151234,0.141001,0.362897,1.726878,0.007112,0.044281,-2.376506,0.153878,2.392119,0.08812,-0.236717,0.016733,-0.034499,0.065807,-0.074531,0.047404,0.141387,-0.074853,-0.038261,-0.020196,0.03234,-0.064777,2.125607,-0.045468,0.04781,-0.06545,-0.000193,-0.429509,0.655512,0.099622,-0.092667,-0.442808,0.019282,0.430884,0.045832,-0.03387,0.0709,0.004135,-0.122761,0.018717,-0.038279,2.01171,2.054469,-0.01099,0.101134,0.031641,-2.199975,0.054284,0.020288,0.137378,2.233222,0.080574,-0.123849,-0.358455,2.085705,-1.983462,0.677206,-0.173973,-0.038308,0.372832,0.326945,-0.291275,-0.065318,0.428629,-2.564597,0.345225,2.088677,-2.335162,-0.113657,0.075214,2.059409,0.525848,-0.430789,0.223353,0.432391,-2.635306,0.246727,0.036412,0.046987,0.135662,0.031599,0.052198,0.432797,-2.128332,-0.216767,0.022602,2.751684,0.087278,-2.967879,2.933561,-0.036041,0.067418,0.213866,-0.007657,-1.841815,1.942968,-0.076189,0.052228,0.149901,-0.007161,-0.064303,0.058158,-0.532908,0.429528,0.405651,-0.070227,0.236214,-0.496152,0.045877,0.066543,-0.041307,0.334081,-0.111002,0.102659,-0.027857,0.13548,-0.020582,-0.104633,0.109376,-0.00774,0.115855,-0.025528,0.038041,-0.022756,0.252013,0.195369,-1.334428,0.618779
2,-3.035199,0.339819,0.101292,-0.867509,-0.352161,0.386487,-0.114646,0.028338,-0.271671,-0.450577,0.453338,0.426511,-2.734621,0.836345,0.234737,-1.853026,-2.031305,-1.088093,1.312025,-0.086444,-0.114457,0.109626,0.053639,-1.476198,1.763554,0.314331,-0.461516,-0.026144,1.93647,-0.066335,0.121528,0.247095,-1.55651,0.03999,0.088676,-2.152806,0.151234,0.141001,0.362897,1.726878,0.007112

In [156]:
tick_ = tick_df.T.median(axis=1)
usd_ = usd_df.T.median(axis=1)
volm_ = volm_df.T.median(axis=1)
calendar_ = calendar_df.T.median(axis=1)


In [124]:
tick_median = df.T.median(axis=1)

0      -0.051025
1      -0.375005
2      -0.258944
3      -0.277096
4       0.258128
          ...   
4430   -0.147337
4431   -0.226205
4432    0.179210
4433   -0.674989
4434   -0.598932
Length: 4435, dtype: float64

In [165]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,10))
sns.kdeplot(tick_, 
            ax=ax,
            label="Tick", 
            bw=0.25,
            color='darkblue')
sns.kdeplot(usd_,
            ax=ax,
            label="USD Volume",
            bw=1.55,linewidth=2.25,
            color='purple', linestyle ='-')

sns.kdeplot(volm_,
            ax=ax,
            label="Volume",
            bw=0.55,linewidth=1.25,
            color='red', linestyle ='-')
sns.kdeplot(calendar_,
            ax=ax,
            label="Calendar",
            bw=0.55,linewidth=2.25,
            color='grey', linestyle =':')

sns.kdeplot(np.random.normal(size=len(dr)),
            ax=ax,
            label="Normal",
            color='darkgreen',
            linewidth=3,
            linestyle="--")
plt.legend()


_ =plt.xlabel('Returns', fontsize=19)
_ =plt.ylabel('Density', fontsize=19)
_ =plt.xticks(fontsize = 15, rotation = 45)
_ = plt.yticks(fontsize = 15)

#plt.title(str(symbol))
file_name = str(symbol) + 'median_multiclocks_density_plot.png'
plt.savefig(os.path.join(figures_destination,file_name))
plt.legend()
plt.show()

/home/ak/anaconda3/envs/mmd-kernels/lib/python3.7/site-packages/seaborn/distributions.py:1657: FutureWarning: The `bw` parameter is deprecated in favor of `bw_method` and `bw_adjust`. Using 0.25 for `bw_method`, but please see the docs for the new parameters and update your code.
  warnings.warn(msg, FutureWarning)
/home/ak/anaconda3/envs/mmd-kernels/lib/python3.7/site-packages/seaborn/distributions.py:1657: FutureWarning: The `bw` parameter is deprecated in favor of `bw_method` and `bw_adjust`. Using 1.55 for `bw_method`, but please see the docs for the new parameters and update your code.
  warnings.warn(msg, FutureWarning)
/home/ak/anaconda3/envs/mmd-kernels/lib/python3.7/site-packages/seaborn/distributions.py:1657: FutureWarning: The `bw` parameter is deprecated in favor of `bw_method` and `bw_adjust`. Using 0.55 for `bw_method`, but please see the docs for the new parameters and update your code.
  warnings.warn(msg, FutureWarning)
/home/ak/anaconda3/envs/mmd-kernels/lib/python3.7